In [0]:
# Databricks Notebook: 00_Mount_Storage
# Run this cell-by-cell using Shift+Enter

# Cell 1: Get credentials from secret scope
storage_account_name = "adlscti20241125"  # CHANGE THIS to your storage account name
client_id = dbutils.secrets.get(scope="cti-scope", key="sp-client-id")
client_secret = dbutils.secrets.get(scope="cti-scope", key="sp-client-secret")
tenant_id = dbutils.secrets.get(scope="cti-scope", key="tenant-id")

print("Credentials retrieved from secret scope")

# Cell 2: Configure authentication
configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": client_id,
  "fs.azure.account.oauth2.client.secret": client_secret,
  "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

print(" Configuration prepared")

# Cell 3: Mount RAW container
try:
    dbutils.fs.mount(
        source = f"abfss://raw@{storage_account_name}.dfs.core.windows.net/",
        mount_point = "/mnt/cti/raw",
        extra_configs = configs
    )
    print(" Mounted: /mnt/cti/raw")
except Exception as e:
    if "already mounted" in str(e).lower():
        print("  Already mounted: /mnt/cti/raw")
    else:
        print(f" Error: {e}")

# Cell 4: Mount BRONZE container
try:
    dbutils.fs.mount(
        source = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/",
        mount_point = "/mnt/cti/bronze",
        extra_configs = configs
    )
    print("Mounted: /mnt/cti/bronze")
except Exception as e:
    if "already mounted" in str(e).lower():
        print(" Already mounted: /mnt/cti/bronze")
    else:
        print(f"Error: {e}")

# Cell 5: Mount SILVER container
try:
    dbutils.fs.mount(
        source = f"abfss://silver@{storage_account_name}.dfs.core.windows.net/",
        mount_point = "/mnt/cti/silver",
        extra_configs = configs
    )
    print(" Mounted: /mnt/cti/silver")
except Exception as e:
    if "already mounted" in str(e).lower():
        print("Already mounted: /mnt/cti/silver")
    else:
        print(f" Error: {e}")

# Cell 6: Mount GOLD container
try:
    dbutils.fs.mount(
        source = f"abfss://gold@{storage_account_name}.dfs.core.windows.net/",
        mount_point = "/mnt/cti/gold",
        extra_configs = configs
    )
    print(" Mounted: /mnt/cti/gold")
except Exception as e:
    if "already mounted" in str(e).lower():
        print(" Already mounted: /mnt/cti/gold")
    else:
        print(f"❌Error: {e}")

# Cell 7: Verify all mounts
print("\n📂 All mount points:")
display(dbutils.fs.mounts())

# Cell 8: Test reading from RAW
print("\n🔍 Files in /mnt/cti/raw/otx:")
try:
    files = dbutils.fs.ls("/mnt/cti/raw/otx")
    for file in files:
        print(f"  - {file.name}")
except Exception as e:
    print(f"  No files yet or error: {e}")